In [5]:
import pandas as pd

# Reload the corrected dataset
df = pd.read_csv("North Group CSV.csv", low_memory=False)
df.columns = df.columns.str.strip()

# Preprocess team names
df[['Team1', 'Team2']] = df['Match'].str.split(' v ', expand=True)
df.sort_values(by=['Match', 'Date', 'Innings'], inplace=True)
df['Next_Innings'] = df.groupby(['Match', 'Date'])['Innings'].shift(-1)
mask = df['Innings'].ne(df['Next_Innings']) & (df.index < len(df))
rows_before_change = df[mask].copy().reset_index(drop=True).drop(columns=['Next_Innings'])

# Helper functions
def cricket_overs_to_balls(overs):
    overs_int = int(overs)
    balls_part = int(round((overs - overs_int) * 10))
    return overs_int * 6 + balls_part

def balls_to_cricket_overs(balls):
    overs = balls // 6
    rem_balls = balls % 6
    return float(f"{int(overs)}.{int(rem_balls)}")

def corrected_actual_overs(row):
    adjusted_ball = row['Actual Ball']
    if row['Legal Ball'] != 'Yes':
        adjusted_ball = max(0, adjusted_ball - 1)
    if adjusted_ball == 6:
        return float(f"{int(row['Over'])}.0")
    else:
        return float(f"{int(row['Over']) - 1}.{int(adjusted_ball)}")

# Calculate overs and run rates
rows_before_change['Actual Overs'] = rows_before_change.apply(corrected_actual_overs, axis=1)
rows_before_change['NRR Overs'] = rows_before_change.apply(
    lambda row: 20.0 if row['Team Wickets'] == 10 else row['Actual Overs'], axis=1)
rows_before_change['NRR Balls'] = rows_before_change['NRR Overs'].apply(cricket_overs_to_balls)

# Manually override two rain-shortened first innings scores
rain_shortened_scores = {
    ("Derbyshire Falcons Men", "06/06/2025"): (119, 15.0),
    ("Worcestershire Rapids Men", "01/06/2025"): (195, 19.3)
}

for (team, date), (runs, overs) in rain_shortened_scores.items():
    idx = rows_before_change[
        (rows_before_change['Batting Team'] == team) & (rows_before_change['Date'] == date)
    ].index
    rows_before_change.loc[idx, 'Team Runs'] = runs
    rows_before_change.loc[idx, 'NRR Overs'] = overs
    rows_before_change.loc[idx, 'NRR Balls'] = cricket_overs_to_balls(overs)

# Recalculate team summaries
for_summary = rows_before_change.groupby('Batting Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Batting Team': 'Team',
    'Team Runs': 'Runs For',
    'NRR Balls': 'NRR Balls For'
})
for_summary['Overs For'] = for_summary['NRR Balls For'].apply(balls_to_cricket_overs)
for_summary.drop(columns='NRR Balls For', inplace=True)

against_summary = rows_before_change.groupby('Bowling Team').agg({
    'Team Runs': 'sum',
    'NRR Balls': 'sum'
}).reset_index().rename(columns={
    'Bowling Team': 'Team',
    'Team Runs': 'Runs Against',
    'NRR Balls': 'NRR Balls Against'
})
against_summary['Overs Against'] = against_summary['NRR Balls Against'].apply(balls_to_cricket_overs)
against_summary.drop(columns='NRR Balls Against', inplace=True)

team_summary = pd.merge(for_summary, against_summary, on='Team', how='outer')
team_summary['NRR Balls For'] = team_summary['Overs For'].apply(cricket_overs_to_balls)
team_summary['NRR Balls Against'] = team_summary['Overs Against'].apply(cricket_overs_to_balls)
team_summary['Run Rate For'] = team_summary['Runs For'] / (team_summary['NRR Balls For'] / 6)
team_summary['Run Rate Against'] = team_summary['Runs Against'] / (team_summary['NRR Balls Against'] / 6)
team_summary['NRR'] = (team_summary['Run Rate For'] - team_summary['Run Rate Against']).round(3)
final_nrr_table = team_summary[['Team', 'Runs For', 'Overs For', 'Runs Against', 'Overs Against', 'NRR']]

# Match results W/L/T/N/R logic
match_results = []
innings_grouped = rows_before_change.groupby(['Match', 'Date'])
for (match, date), group in innings_grouped:
    if len(group) == 2:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[1]['Batting Team']
        runs1 = group.iloc[0]['Team Runs']
        runs2 = group.iloc[1]['Team Runs']
        if runs1 > runs2:
            match_results.extend([
                {'Team': team1, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
            ])
        elif runs1 == runs2:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
                {'Team': team2, 'W': 0, 'L': 0, 'T': 1, 'N/R': 0},
            ])
        else:
            match_results.extend([
                {'Team': team1, 'W': 0, 'L': 1, 'T': 0, 'N/R': 0},
                {'Team': team2, 'W': 1, 'L': 0, 'T': 0, 'N/R': 0},
            ])
    else:
        team1 = group.iloc[0]['Batting Team']
        team2 = group.iloc[0]['Bowling Team']
        match_results.extend([
            {'Team': team1, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
            {'Team': team2, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1},
        ])
results_df = pd.DataFrame(match_results)
summary_stats = results_df.groupby('Team').sum().reset_index()
summary_stats['M'] = summary_stats[['W', 'L', 'T', 'N/R']].sum(axis=1)
summary_stats['PT'] = summary_stats['W'] * 4 + summary_stats['T'] * 2 + summary_stats['N/R'] * 2

# Add manually abandoned game (Durham v Leicestershire on 15 July 2025)
abandoned = ("Durham Cricket Men", "Leicestershire Foxes Men")
for team in abandoned:
    if team in summary_stats['Team'].values:
        summary_stats.loc[summary_stats['Team'] == team, 'M'] += 1
        summary_stats.loc[summary_stats['Team'] == team, 'N/R'] += 1
        summary_stats.loc[summary_stats['Team'] == team, 'PT'] += 2
    else:
        summary_stats = pd.concat([
            summary_stats,
            pd.DataFrame([{
                'Team': team, 'W': 0, 'L': 0, 'T': 0, 'N/R': 1, 'M': 1, 'PT': 2
            }])
        ], ignore_index=True)

# Merge and prepare final standings
final_merged = pd.merge(summary_stats, final_nrr_table, on='Team', how='outer')
final_display = final_merged[
    ['Team', 'M', 'W', 'L', 'T', 'N/R', 'PT', 'NRR', 'Runs For', 'Overs For', 'Runs Against', 'Overs Against']
].sort_values(by=['PT', 'NRR'], ascending=[False, False]).reset_index(drop=True)
final_display.index += 1
final_display.to_csv("current_table.csv", index=False)
final_display

,Team,M,W,L,T,N/R,PT,NRR,Runs For,Overs For,Runs Against,Overs Against
1,Lancashire Lightning,12,8,4,0,0,32,0.535,2132,235.4,2030,238.3
2,Northamptonshire Steelbacks Men,13,8,5,0,0,32,0.207,2367,259.2,2251,252.2
3,Durham Cricket Men,13,7,5,0,1,30,0.598,2012,235.1,1866,234.3
4,Bears Men,13,7,6,0,0,28,0.228,2343,256.5,2280,256.2
5,Leicestershire Foxes Men,13,6,6,0,1,26,-0.211,1939,228.1,2035,233.4
6,Worcestershire Rapids Men,13,6,7,0,0,24,0.057,2160,251.1,2167,253.4
7,Notts Outlaws,12,6,6,0,0,24,-0.565,2080,231.3,2222,232.4
8,Yorkshire Men,12,4,8,0,0,16,-0.383,2162,239.3,2169,230.3
9,Derbyshire Falcons Men,13,4,9,0,0,16,-0.510,2209,247.2,2384,252.3
